In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import os
from datetime import timedelta
from IPython.display import Image,display
import prettytensor as pt ## For simplifying neuralnetwork representation
import inception  ### A python file in the directory

In [ ]:
import cifar10 ## MOdule in the directory
from cifar10 import num_classes
cifar10.data_path = "data/CIFAR-10"

In [ ]:
cifar10.maybe_download_and_extract() ### A function in cifar10 module

In [ ]:
class_names = cifar10.load_class_names()

In [ ]:
class_names ## Names in cifar10 dataset

In [ ]:
images_train, cls_train, labels_train = cifar10.load_training_data()

In [ ]:
images_test, cls_test, labels_test = cifar10.load_test_data()

In [ ]:
print("Size of--")
print("Training Data \t\t{}".format(len(images_train)))
print("Test Data \t\t{}".format(len(images_test)))

In [ ]:
def plot_images(images, cls_true, cls_pred=None, smooth=True):
    
    assert len(images)==len(cls_true)
    
    fig, axes = plt.subplots(3,3)
    
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6 ## More hspace to represent since we need to represnt both cls_pred and cls_true
    
    if smooth is True:
        interpolation = "spline16"  
    else:
        interpolation = "nearest"
        
    for i, ax in enumerate(axes.flat):
        
        if i<len(images):  ### VIMP---> in case images are less than 9
            ax.imshow(images[i], interpolation=interpolation)  ## Each ax is drawn
        
            cls_true_name = class_names[cls_true[i]]
            
            if cls_pred is None: ## Only 1 thing to represent
                xlabel = "True:{0}".format(cls_true_name)
            else:   ## 2 things two represent
                cls_pred_name = class_names[cls_pred[i]]
                xlabel = "True:{0}\nPredicted:{1}".format(cls_true_name,cls_pred_name)
        
            ax.set_xlabel(xlabel) 
        
        ax.set_xticks([]) ### Remove xticks from plot
        ax.set_yticks([]) ### Remove yticks from plot
    plt.show()


In [ ]:
##Show some images
images = images_train[0:9]

cls_true = cls_train[0:9]

plot_images(images,cls_true,cls_pred=None,smooth=True)

In [ ]:
inception.data_dir = 'inception/' ##Directory would be created if not present

In [ ]:
inception.maybe_download() ## It would be downlaode if not

In [ ]:
model = inception.Inception() ## Model is loaded

In [ ]:
### Inception Model----> We will not train our Inception Model on Cifar10 dataset. Instead we are using the same trained 
## Inception model and altering our output wrt to Cifar10
## Our last layer is called transfer layer and its output is transfer values which we will store in cache files so as to ease 
## the access. Now all these transfer values are inputted to another neural network(II). Whose final layer is a softmax layer 
## with 10outputs for each of the classes

In [ ]:
from inception import transfer_values_cache ## A function to convert transfer values to cache

In [ ]:
## File to store cache 
file_path_cache_train = os.path.join(cifar10.data_path,'inception_cifar10_train.pkl')
file_path_cache_test = os.path.join(cifar10.data_path, 'inception_cifar10_test.pkl')

In [ ]:
print("Processing Transfer values for training Images--->")
### Images are scaled since cifar10 is returning pixel between 1 and 10 and we need in between 0 and 255
images_scaled = images_train*255.0

transfer_values_train = transfer_values_cache(cache_path=file_path_cache_train,
                                             images=images_scaled,
                                             model=model)

In [ ]:
print("Processing Transfer values for testing images--->")

images_scaled = images_train * 255.0

transfer_values_test = transfer_values_cache(cache_path = file_path_cache_test,
                                            images = images_scaled,
                                            model = model)

## We are calculating transfer values and saving them as cache on the file

In [ ]:
transfer_values_train.shape
## Shape is (50000,2048) i.e. each images has an array 0f 2048 elements which are basically output from transfer value

In [ ]:
transfer_values_test.shape
## Shape is (10000,2048)

In [ ]:
## Function to plot transfer values for test
def plot_transfer_values(i):
    print("Input Image:")
    
    plt.imshow(images_test[i],interpolation='nearest') ## Plots input test image 32*32 pixel hence pixelated
    plt.show()
    
    print("Transfer values for image using Inception Model:")
    img = transfer_values_test[i]
    img = img.reshape((32,64))
    plt.imshow(img,interpolation='nearest',cmap'Reds') ##cmap-color map image plotted would be of fifferent degrree of red color
    plt.show()
    
## Transfer values are plotted cannot be understood by normal means

In [ ]:
## Now we have to analyse transfer values so as to see of inception model is successfull to sepearte usefull information and 
## seperate classes. But transfer values have 2048 and hence difficult to plot 
## Hence we do dimensionality reduction is applied

from sklearn.decomposition import PCA

pca = PCA(n_components=2) ## N0componets decide to which level to be dropped

transfer_values = transfer_values_train[0:3000] ## Only reducing first 3000 images since takes a lot of time to convert

cls = cls_train[0:3000]

transfer_values.shape ## Right now shape is 3000,2048

In [ ]:
transfer_values_reduced = pca.fit_transform(transfer_values)

transfer_values_reduced.shape ## Shape is 3000,2  Shape is reuced to 2

In [ ]:
def plot_scatter(values,cls):
    
    import matplotlib.cm as cm
    ## By this we can define our own colormap differnet for each class[10 classes]
    cmap = cm.rainbow(np.linspace(0.0,1.0,num_classes))
    ## Color map created np.linspace defines equally spced 10 numbers between 0 to 1
    colors = cmap[cls] ## get color for each cls
    x = values[:,0] ## First value
    y = values[:,1]  ## Second value
    
    plt.scatter(x,y,colors=colors) ## Scatter plot
    plt.show()

In [ ]:
plot_scatter(transfer_values_reduced,cls) ## Plotting reduce dtransfer values
## Values have been plotted and though there have been seperation of classes but still there is large large overlapping of 
## classes which can be due to several resaons---maybe PCA is not able to reduce 2048 to 2 or maybe Incetion model is not able 
## to differentiate between classes

In [ ]:
### Now using a more advanced technique--> t-SNE  t-distributed Stochastic Neighbor Embedding
## It can effectively reduce high diensions into saller with a preconditionthat initiall another dimensionalyt reduction has
## to be applied first

from sklearn.manifold import TSNE

pca = PCA(n_components=50) ## pca applied and reuce dimension to 50
transfer_values50d = pca.fit_transform(transfer_values) ## now dimensions are 3000,50

tsne = TSNE(n_components=2)
transfer_values_reduced = tsne.fit_transform(transfer_values50d)

transfer_values_reduced.shape ## Shape is 3000,2

In [ ]:
plot_scatter(transfer_values_reduced,cls) ## Now plot show much better sepearated clusters which tells us that transfer values
## can indeed be used to seperate between classes

In [ ]:
### Now since transfer values are capable of seperating classes
## We need to construct a neural network that takes transfer values as input and output class numbers

In [ ]:
transfer_len = model.transfer_len ## Array length of transfer values=2048

In [ ]:
x = tf.placeholder(tf.float32, shape = [None,transfer_len],name='x') ## None represent no of images which will be inputted
                                                                    
y_true = tf.placeholder(tf.float32, shape = [None,num_classes], name='y_true') ## true labels that are inputted

y_true_cls = tf.argmax(y_true,axis=1) ## True classes in int form

In [ ]:
## Creating Neural Network using pretty tensor
x_pretty = pt.wrap(x)

with pt.defaults_scope(activation_fn=tf.nn.relu):  ## i.e BY default all ndes have relu activation function
    y_pred,loss = x_pretty.\ ## IMP----> output is y_pred & loss
        fully_connected(size=1024,name='layer_fc1').\
        softmax_classifier(num_classes=num_classes,labels=y_true)

In [ ]:
global_step = tf.Variable(initial_value=50,
                         name='global_step',trainable=False) ## It means during optimization this will not train
## This is used only as a step counter

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss,global_step)

In [ ]:
y_pred_cls = tf.argmax(y_pred,axis=1) ## y_pred is in hot encode form this will be now in int form

In [ ]:
correct_prediction = tf.equal(y_pred_cls,y_true_cls)  ## This will yield a boolean array

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) 
### IMP--> reduce_mean basically calculates average tf.cas boolean array into float

In [ ]:
session  = tf.Session() ## Creates a tensorflow session
session.run(tf.global_variables_initializer()) ## Variables initialized

In [ ]:
## WE will be using batch gradient descent i.e instead of going thro' all the examples in trainig data
## to make one gradient descent step we will make a step once going thro' a batch of training data

train_batch_size = 64

def random_batch():
    num_images = len(transfer_values_train) ## IMP---> Transfer balues are input 
    
    idx = np.random.choice(num_images,
                          size = train_batch_size,  ## Out of total images how manny images has to be chosen randomly
                          replace = False) ## This means same image will not repeated
    x_batch = transfer_values_train[idx]
    y_batch = labels_train[idx]
    
    return x_batch,y_batch

In [ ]:
def optimize(num_iterations):
    
    start_time = time.time() ###For time usage
    
    for i in range(num_iterations):
        
        x_batch, y_true_batch = random_batch()
        
        feed_dict_train = { x : x_batch, y_true : y_true_batch}
        
        i_global,_ = session.run([global_step,optimizer],feed_dict=feed_dict_train)
        ### we are running global_step also since we need global step counter i.e i+global
        
        if (i_global %100==0) or (i ==num_iterations-1):
            
            batch_acc = session.run(accuracy,feed_dict=feed_dict_train)
            ## Accuracy tensorflow graph is run and it's batch accuracy is stored in batch_acc
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))
            
    end_time = time.time()
    
    time_dif = end_time - start_time
    
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
            

In [ ]:
def plot_example_errors(cls_pred, correct):
    
    incorrect = (correct == False) ### Negate the boolean array
    
    images = images_test[incorrect] ## Images of test set that have been incorrectly classified
    cls_pred = cls_pred[incorrect]  ## Predicted classes of incorrectly classified images
    cls_true = cls_test[incorrect] ## True classes of incorrectly classified images
    n = min(9,len(images))
    ## Using abve function to plot first n images
    plot_images(images=images[0:n],cls_true=cls_true[0:n],cls_pred=cls_pred[0:n])

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cls_pred):
    
    cm = confusion_matrix(y_true = cls_test,
                         y_pred = cls_pred)
    
    for i in range(num_classes):
        
        class_name = "({}) {}".format(i,class_names[i])
        print(cm[i,:],class_name)
        
    class_numbers = [" ({0})".format(i) for i in range(num_classes)]
    print("".join(class_numbers))

In [ ]:
batch_size = 256

def predict_cls(transfer_values, labels, cls_true):
    num_images = len(transfer_values)
    
    cls_pred = np.zeros(shape=num_images, dtype=np.int)
    i = 0

    while i < num_images:
        j = min(i + batch_size, num_images)
        
        feed_dict = {x: transfer_values[i:j],
                     y_true: labels[i:j]}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)
    
    return correct, cls_pred

In [ ]:
def predict_cls_test():
    return predict_cls(transfer_values = transfer_values_test, ##VIMP---> pass transfer values
                       labels = labels_test,
                       cls_true = cls_test)

In [ ]:
def classification_accuracy(correct): ## Correct is aboolean array
    return correct.mean(), correct.sum() ## Mean is basically accuracy only

In [ ]:
## Fucntion for printing test accuracy
def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    correct, cls_pred = predict_cls_test()
    
    # Classification accuracy and the number of correct classifications.
    acc, num_correct = classification_accuracy(correct)
    
    # Number of images being classified.
    num_images = len(correct) ## VIMP---> This show total images coorect as wella sincorrect

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [ ]:
optimize(num_iterations = 10000) ## Num of iterations

In [ ]:
print_test_accuracy(show_example_errors=True,show_confusion_matrix=True)
## This will plot some images that are incorreclty misclassified and confusion matrix


In [ ]:
## Giving an accuracy of-------> ""90.7%"" at 10000 iterations as compared to using normal convolutional neural networks 
### which gives only 80%at 15000 iterations